In [2]:
import re
import pandas as pd

# 1. Создаем правила разметки

In [3]:
def label_post(post):
    depressed_pattern = r'\b(sad|lonely|hopeless|depressed|bad)\b'
    if re.search(depressed_pattern, post['body'], re.IGNORECASE) or (post['num_comments'] > 10 and post['upvotes'] < 5):
        return 1 #depressed
    return 0 #normal

In [4]:
data = pd.read_csv('reddit_depression_dataset.csv')
data.head()

C:\Users\igor9\AppData\Local\Temp\ipykernel_24128\96955893.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('reddit_depression_dataset.csv')


,Unnamed: 0,subreddit,title,body,upvotes,created_utc,num_comments,label
0,47951,DeepThoughts,Deep thoughts underdog,"Only when we start considering ourselves, the ...",4.0,1.405309e+09,NaN,0.0
1,47952,DeepThoughts,"I like this sub, there's only two posts yet I ...",Anyway: Human Morality is a joke so long as th...,4.0,1.410568e+09,1.0,0.0
2,47957,DeepThoughts,Rebirth!,Hello. \nI am the new guy in charge here (Besi...,6.0,1.416458e+09,1.0,0.0
3,47959,DeepThoughts,"""I want to be like water. I want to slip throu...",NaN,25.0,1.416512e+09,2.0,0.0
4,47960,DeepThoughts,Who am I?,You could take any one cell in my body and kil...,5.0,1.416516e+09,4.0,0.0


In [5]:
data = data.dropna()
data.head()

,Unnamed: 0,subreddit,title,body,upvotes,created_utc,num_comments,label
1,47952,DeepThoughts,"I like this sub, there's only two posts yet I ...",Anyway: Human Morality is a joke so long as th...,4.0,1.410568e+09,1.0,0.0
2,47957,DeepThoughts,Rebirth!,Hello. \nI am the new guy in charge here (Besi...,6.0,1.416458e+09,1.0,0.0
4,47960,DeepThoughts,Who am I?,You could take any one cell in my body and kil...,5.0,1.416516e+09,4.0,0.0
5,47969,DeepThoughts,What is the limit of the knowledge and power a...,"Personally, I think it's infinite. We will alw...",8.0,1.416684e+09,23.0,0.0
7,47973,DeepThoughts,"10,000 Bits Contest (ChangeTip)","Hello, from now to Christmas we will be hostin...",7.0,1.416870e+09,26.0,0.0


In [7]:
data['label'] = data.apply(label_post, axis=1)
data.to_csv('reddit_posts_labeled_rule_based.csv', index=False)

# 2. Создаем отдельный файл для разметки вручную

In [8]:
manual_labeling_data = data.sample(n=100)
manual_labeling_data.to_csv('manual_labeling_data.csv')

# 3. Объединение данных размеченных вручную с данными, размеченными по правилу

In [10]:
manual_labels = pd.read_csv('reddit_posts_labled_manualy.csv')
rule_based_labels = pd.read_csv('reddit_posts_labeled_rule_based.csv')

In [11]:
combined_data = pd.concat([manual_labels, rule_based_labels], ignore_index=True)
combined_data.to_csv('combined_data.csv')

# 4. Обучение модели

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

In [14]:
data = pd.read_csv('combined_data.csv')

C:\Users\igor9\AppData\Local\Temp\ipykernel_24128\1280830815.py:1: DtypeWarning: Columns (6,12,15) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('combined_data.csv')


In [16]:
x = data['body']
y = data['label']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

model = make_pipeline(CountVectorizer(), LogisticRegression(max_iter=500))
model.fit(x_train, y_train)

e:\_Disk\gb.ru\Разработчик - DataEngineer\Сбор и разметка данных\venv\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('logisticregression', LogisticRegression(max_iter=500))])

# 5. Оценка эффективности модели

In [17]:
from sklearn.metrics import classification_report

In [18]:
y_pred = model.predict(x_test)

classification_report(y_test, y_pred)

'              precision    recall  f1-score   support\n\n           0       0.95      1.00      0.97    316730\n           1       1.00      0.75      0.86     68376\n\n    accuracy                           0.96    385106\n   macro avg       0.97      0.88      0.92    385106\nweighted avg       0.96      0.96      0.95    385106\n'

# Оценка результатов
## Сильные стороны:

- Высокая точность и полнота для нормальных постов.
- Идеальная точность для депрессивных постов.
- Общая высокая точность и хорошие F1-меры.
## Области для улучшения:

- Полнота для депрессивных постов (75%) может быть улучшена, чтобы уменьшить количество пропущенных случаев. Это особенно важно, если модель используется для обнаружения потенциально опасных состояний у пользователей.